In [ ]:
%pip install ../dist/alquimia_fair_forge-0.0.1.tar.gz -q

In [ ]:
from fair_forge.metrics import Humanity
from helpers.retriever import LocalRetriever
from elasticsearch import Elasticsearch, helpers
import os

In [3]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
index_name = f"test-humanity"

In [ ]:
metrics = Humanity.run(LocalRetriever,verbose=True)

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [3]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
mapping_humanity = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "humanity_assistant_emotional_entropy": {"type": "float"},
      "humanity_ground_truth_spearman": {"type": "float"},
      "humanity_assistant_anger": {"type": "float"},
      "humanity_assistant_anticipation": {"type": "float"},
      "humanity_assistant_disgust": {"type": "float"},
      "humanity_assistant_fear": {"type": "float"},
      "humanity_assistant_joy": {"type": "float"},
      "humanity_assistant_sadness": {"type": "float"},
      "humanity_assistant_surprise": {"type": "float"},
      "humanity_assistant_trust": {"type": "float"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}
recreate_index(index_name, mapping_humanity)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": index_name,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")